## 201 - Lapse Rate Rules of Thumb Part 2

[Youtube](https://www.youtube.com/watch?v=1GkZYf_9-Qs)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from metpy.units import units, pandas_dataframe_to_unit_arrays
from siphon.simplewebservice.iastate import IAStateUpperAir

In [2]:
year = 2020
datetimes_to_get = []
for month in range(1, 13):
	for time in [0, 12]:
		datetimes_to_get.append(datetime(year, month, 1, time))

In [3]:
df = IAStateUpperAir.request_all_data(datetimes_to_get.pop())

In [4]:
for dt in datetimes_to_get:
	print(f'Getting {dt}')
	df = pd.concat([df, IAStateUpperAir.request_all_data(dt)])

Getting 2020-01-01 00:00:00
Getting 2020-01-01 12:00:00
Getting 2020-02-01 00:00:00
Getting 2020-02-01 12:00:00
Getting 2020-03-01 00:00:00
Getting 2020-03-01 12:00:00
Getting 2020-04-01 00:00:00
Getting 2020-04-01 12:00:00
Getting 2020-05-01 00:00:00
Getting 2020-05-01 12:00:00
Getting 2020-06-01 00:00:00
Getting 2020-06-01 12:00:00
Getting 2020-07-01 00:00:00
Getting 2020-07-01 12:00:00
Getting 2020-08-01 00:00:00
Getting 2020-08-01 12:00:00
Getting 2020-09-01 00:00:00
Getting 2020-09-01 12:00:00
Getting 2020-10-01 00:00:00
Getting 2020-10-01 12:00:00
Getting 2020-11-01 00:00:00
Getting 2020-11-01 12:00:00
Getting 2020-12-01 00:00:00


In [5]:
df

,pressure,height,temperature,dewpoint,direction,speed,station,time,u_wind,v_wind
0,969.0,386.0,-15.7,-17.5,160.0,5.0,CWPL,2020-12-01 12:00:00,-1.710101,4.698463
1,941.6,609.0,NaN,NaN,215.0,19.0,CWPL,2020-12-01 12:00:00,10.897952,15.563889
2,925.0,747.0,-3.9,-24.9,235.0,21.0,CWPL,2020-12-01 12:00:00,17.202193,12.045105
3,917.0,816.0,-1.7,-24.7,NaN,NaN,CWPL,2020-12-01 12:00:00,NaN,NaN
4,913.0,851.0,-1.7,-25.7,NaN,NaN,CWPL,2020-12-01 12:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13010,10.4,30024.0,-64.3,-94.3,NaN,NaN,KYMW,2020-12-01 00:00:00,NaN,NaN
13011,10.3,30083.0,-64.3,-94.3,NaN,NaN,KYMW,2020-12-01 00:00:00,NaN,NaN
13012,10.0,30270.0,-64.5,-92.5,255.0,59.0,KYMW,2020-12-01 00:00:00,56.989624,15.270324
13013,9.7,30456.0,-64.5,NaN,NaN,NaN,KYMW,2020-12-01 00:00:00,NaN,NaN


In [6]:
def calculate_freezing_rule(df):
	idx = np.argmax(df['pressure'].values)
	sfc_temp = df['temperature'].values[idx]
	pred = sfc_temp / 2 * 1000
	return max(0, pred)
	
def calculate_freezing_observed(df):
	# Check if the surface is already freezing
	sfc_idx = np.argmax(df['pressure'].values)
	# print(sfc_idx, df['temperature'].values[sfc_idx])
	if df['temperature'].values[sfc_idx] <= 0:
		return 0
		
	closest_idx = np.argmin(np.abs(df['temperature']))
	start_fit_idx = max(closest_idx - 2, 0)
	end_fit_idx = min(len(df) - 1, closest_idx + 2)
	coeffs = np.polyfit(df['temperature'].values[start_fit_idx: end_fit_idx], df['height'].values[start_fit_idx: end_fit_idx], 1)
	height_est_from_obs = (np.polyval(coeffs, 0) * units.m)
	sfc_idx = np.argmax(df['pressure'].values)
	sfc_elevation = df['height'].values[sfc_idx] * units.m
	return (height_est_from_obs - sfc_elevation).to('feet').m

In [7]:
df_results = pd.DataFrame(columns = ['Time', 'Station', 'Rule', 'Observed'])
for time in df['time'].unique():
	print(time)
	df_time = df[df['time'] == time]
	for stn in df['station'].unique():
		df_station = df_time[df_time['station'] == stn]
		df_station = df_station.dropna(subset = ['temperature', 'height'])
		if len(df_station) == 0:
			continue
		rule_prediction = calculate_freezing_rule(df_station)
		observed = calculate_freezing_observed(df_station)
		if observed > 0:
			df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)

2020-12-01T12:00:00.000000000


/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method

2020-01-01T00:00:00.000000000


/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method

2020-01-01T12:00:00.000000000


...

/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method

2020-11-01T12:00:00.000000000


/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method

2020-12-01T00:00:00.000000000


/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'Time': time, 'Station': stn, 'Rule': rule_prediction, 'Observed': observed}, ignore_index = True)
/tmp/ipykernel_1285961/1081998344.py:13: FutureWarning: The frame.append method

In [8]:
df_results

,Time,Station,Rule,Observed
0,2020-12-01 12:00:00,CWQI,7400.0,10719.731438
1,2020-12-01 12:00:00,CYJT,2300.0,2595.307380
2,2020-12-01 12:00:00,CYZV,1200.0,8573.749166
3,2020-12-01 12:00:00,KADQ,2100.0,2967.159283
4,2020-12-01 12:00:00,KALY,6900.0,8418.376400
...,...,...,...,...
2211,2020-12-01 00:00:00,KVEF,8600.0,9887.382357
2212,2020-12-01 00:00:00,KWAL,9300.0,9314.128256
2213,2020-12-01 00:00:00,KXMR,10000.0,12644.537556
2214,2020-12-01 00:00:00,KYMW,800.0,1648.060851


In [9]:
df_results['error'] = df_results['Rule'] - df_results['Observed']

In [10]:
import matplotlib.pyplot as plt

In [11]:
error_00z = []
error_12z = []
for time in df_results['Time'].unique():
	if datetime.utcfromtimestamp(time.tolist() / 1e9).hour == 0:
		df_time = df_results[df_results['Time'] == time]
		errors_00z.append(df_time['error'].values)
	elif datetime.utcfromtimestamp(time.tolist / 1e9) == 12:
		df_time = df_results[df_results['Time'] == time]
		errors_12z.append(df_time['error'].values)

TypeError: unsupported operand type(s) for /: 'builtin_function_or_method' and 'float'

In [ ]:
fig, ax = plt.subplots(figsize = (12, 10))
ax.violinplot(errors_00z)
ax.violinplot(errors_12z)
ax.set_xticks(np.arange(1, 13))
ax.set_xticklabels(np.arange(1, 13))
ax.set_ylabel('Feet')
ax.set_xlabel('Month')
plt.title('Freezing Level Estimate Error')
ax.achline(y = 0, linestyle = '--', color = 'k', linewidth = 2)